In [1]:
import tensorflow as tf
from eipy.deep_ei import EnsembleIntegration
import numpy as np
from tensorflow.keras.callbacks import TensorBoard

2023-11-15 21:27:18.462787: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 21:27:18.462846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 21:27:18.462879: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 21:27:18.469926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
y_train = np.where(y_train != 1, 0, y_train)
y_test = np.where(y_test != 1, 0, y_test)

In [5]:
import pickle as pkl
'''with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()'''
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [6]:
sample_size = 1000
X_sample = X_test_resized[:sample_size]
y_sample = y_test[:sample_size]

In [7]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
base_predictors = {
    "ResNet50" : res_model,
    "MobileNet" : net_model
}

In [10]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=3,
                        n_samples=1,
                        sampling_aggregation=None,
                        sampling_strategy=None,
                        n_jobs=1,# -1 could be messing up for one bp
                        metrics=metrics,
                        random_state=38,
                        project_name="images",
                        model_building=False,
                        )

In [11]:
EI.fit_base(X_sample, y_sample, modality_name="shmimage")

Training base predictors on shmimage...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%

6/6 [==============================] - 0s 78ms/step


Generating ensemble training data: |█████     | 50%

6/6 [==============================] - 0s 77ms/step


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%

16/16 [==============================] - 6s 345ms/step


Generating ensemble test data: |█████     | 50%

16/16 [==============================] - 1s 87ms/step


Generating ensemble test data: |██████████|100%

In [12]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [27]:
X = np.array([[0,0], [1,0], [0,1], [0,0]])
y_yes = np.array([[0],[1],[1],[0]])
y_no = y_yes ^ 1

In [28]:
model = Sequential([
    Dense(units=2, input_dim=2, activation="sigmoid"),
    Dense(units=1, activation="sigmoid")
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(X, y_no, epochs=10000, verbose=0)

In [29]:
model.predict(X)

1/1 [==============================] - 0s 32ms/step


array([[0.9874643 ],
       [0.01504581],
       [0.01958418],
       [0.9874643 ]], dtype=float32)

xor gate as nn and xnor gate as nn have same architecture. if we're training on one data set (x + labels) and testing on the other, the performance should be 0% if the instances are different and mixed if the same.